# Futures Contracts

<specifications>

A *futures contract* price of an asset at time $T$ is $S(T)$ is given by the formula:

\begin{align}
\mathbb{E}[S(T) \mid F_{t}]
\end{align}

A long position in a futures contract is an agreement to receive a *cash flow* (either postive or negative) from changes inthe futures price during the time the position is held. A short position receives the opposite cashflow. 

<u>Specifications</u>

**The Asset**

When the asset is a commodity, there may be quite a variation in the quality of what is available in the market place. The asset's grade must be specified. The *Intercontinental Exchange* (ICE) has specified the asset in its orange juice futures contract that are US Grade A with Brix value not less than 62.5 degrees. For some commodities a range of grades can be delivered with higher grades receiving a premium to lower grades.

Financial assets in futures contracts are generally well defined and unambiguous. There is no need to specify the grade of S&P 500 futures contract. For treasury futures, there are some interesting features of the Treasury bond and Treasury note futures contrat on the Chicago Board of Trade. The underlying asset in the Treasury bond contract is any treasury bond that has a maturity between 15 and 25 years. In the Treasury note futures contract, the maturity can be between 6.5 and 10 years. In both cases, the exchange has a formula for adjusting the price received according to the coupon and maturity of the bond delivered. 

**The Contract Size**

- The contract size specifies the amount of the asset that has to be delivered under one contract. The correct size of the contract depends on the likely user. Whereas the value of agricultural futures contracts might be $\$10,000 \ \text{to} \$20,000$, it is *much higher* for financial futures. For treasury bond futures contracts, the notional size is $\$100,000$.

**Delivery Months**

- A futures contrac is referred to by its delivery month. The exchange must specifiy the precise period during the month when the delivery can be made. For many futures contracts, the delivery period can be the whole month. 

| Calendar Month | January | February | March | April | May | June | July | August | September | October | November | December |
| --------------- | ------- | -------- | ----- | ----- | --- | ---- | ---- | ------ | --------- | ------- | -------- | -------- |
| Expiration Code | F       | G        | H     | J     | K   | M    | N    | Q      | U         | V      | X        | Z        |

- The exchange specifies the first day trading in a particular month's contract can begin and similarly specifies the last day trading may take place for a given contract. Trading usually ceases a few days before the last day on which delivery can be made. 


## Hedging Strategies Using Futures



Perfect heding situations are rare for a few reasons so there is **basis risk.**

\begin{align}

Basis = S_{t} - F_{s}(\tau,T)

\end{align}

- The equation may be switched- particulary if the asset being hedged is a financial asset. If the underlying asset in the futures contract is the same as the spot price asset the basis converges to $0$.

## Cross Hedging

- If you need to hedge against price movements of an asset purchased in your portfolio and there is not a specific contract that underlying is the same you must choose a contract that is closely related to that asset. In order to determine the *hedge ratio* - the sensitivity of the asset price with respect to a one-unit change in the hedged asset. 

\begin{align}

r_{s} = \alpha + \beta_{f} +\varepsilon

\end{align}

The number of contracts used to hedge a **real asset**:

\begin{align}

N^{\star} = \beta_{F}\frac{Q_{a}}{Q_{F}} \\

Q_{a} \ \text{and} \ Q_b \ \text{are the sizes of your positions}

\end{align}

The number of contracts used to hedge an equity portfolio using **index futures**:

\begin{align}

N^{\star} = \beta_{F}\frac{V_{a}}{V_{F}} \\

V_{a} \ \text{and} \ V_{F} \ \text{are the notional values of each portfolio of assets}

\end{align}
